second version of training that uses aggregation mlp

In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
class InductiveLPDataset(DisjointInductivePathDataset):
    """An inductive link prediction dataset for the ILPC 2022 Challenge."""

    
    
    
    def __init__(self , **kwargs):
        """Initialize the inductive link prediction dataset.

        :param size: "small" or "large"
        :param kwargs: keyword arguments to forward to the base dataset class, cf. DisjointInductivePathDataset
        """
        DATA_TYPE = "_fully_inductive.tsv"
        TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
        TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
        VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE
        INFERENCE_PATH = "MSCallGraph_0_inference" + DATA_TYPE


        super().__init__(
            transductive_training_path=os.getcwd()+"/"+TRAIN_PATH,
            inductive_inference_path=os.getcwd()+"/"+INFERENCE_PATH,
            inductive_validation_path=os.getcwd()+"/"+VALIDATE_PATH,
            inductive_testing_path=os.getcwd()+"/"+TEST_PATH,
            create_inverse_triples=True,
            eager=True,
            **kwargs
        )


In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [4]:
dataset = InductiveLPDataset()

In [5]:
model_name = 'nodepiece_0_inductive_2'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
num_tokens = 20
embedding_dim = 200

In [8]:
model = InductiveNodePiece(
        triples_factory=dataset.transductive_training,
        inference_factory=dataset.inductive_inference,
        random_seed = seed,
        loss = loss,
        num_tokens = num_tokens,
        embedding_dim = embedding_dim,
        aggregation = 'mlp'
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/3.83k [00:00<?, ?it/s]

No symbolic computation of output shape.


sampling:   0%|          | 0.00/1.30k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 1683200
Space occupied: 6732800 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_0_inductive_2 created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        mode=TRAINING,
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        mode=VALIDATION,
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.inductive_inference,
    evaluation_triples_factory = dataset.inductive_validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.transductive_training,
        model=model,
        mode=TRAINING,  # must be specified for the inductive setup
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.transductive_training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.658198833465576
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.014687752578316494
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.022508623034576538
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.018598187806446516
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.014467868953943253
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.02241574414074421
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.018441807478666306
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.014334581470659042
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.022337034149034846
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.018335807809846946
Metric: validation.head.optimistic.hits_at_1 = 0.00558659217877095
Metric: validation.tail.optimistic.hits_at_1 = 0.0022346368715083797
Metric: validation.both.optimistic.hits_at_1 = 0.00

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 15.826227188110352
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.36805760814739025
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.011458219774334331
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.18975791396086222
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00602674437686801
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.01142834685742855
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.008727545849978924
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.00533204680635465
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.011399750444231499
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.008365898625293074
Metric: validation.head.optimistic.hits_at_1 = 0.3642458100558659
Metric: validation.tail.optimistic.hits_at_1 = 0.0011173184357541898
Metric: validation.both.optimistic.hits_at_1 = 0.182681

...

Step: 45
Metric: loss = 0.7402522563934326
Step: 45
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.015477991727732216
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.033738695947138
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.02460834383743511
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.01506538409739733
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.03353705629706383
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.024301221594214436
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.014797348161456765
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.033362385710768515
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.024079866936112637
Metric: validation.head.optimistic.hits_at_1 = 0.0022346368715083797
Metric: validation.tail.optimistic.hits_at_1 = 0.004469273743016759
Metric: validation.both.optimistic.hits_at_1 = 0.003

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 46
Metric: loss = 0.7498058676719666
Step: 46
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.015332514301390389
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.03351109013610448
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.024421802218747433
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.014913049526512623
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.03328694775700569
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.02409999631345272
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.014639519467178091
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.033096666447015075
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.023868092957096583
Metric: validation.head.optimistic.hits_at_1 = 0.0022346368715083797
Metric: validation.tail.optimistic.hits_at_1 = 0.00558659217877095
Metric: validation.both.optimistic.hits_at_1 = 0.0

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 47
Metric: loss = 0.744475245475769
Step: 47
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.015182071938257364
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.03260266417113301
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.02389236805469518
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.014753106981515883
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.03232843801379204
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.02354077436029911
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.014471840326258784
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.03210415765241938
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.02328799898933908
Metric: validation.head.optimistic.hits_at_1 = 0.0022346368715083797
Metric: validation.tail.optimistic.hits_at_1 = 0.00558659217877095
Metric: validation.both.optimistic.hits_at_1 = 0.00391

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
             0
0     4.658199
1    15.826227
2   112.926422
3    37.369450
4     9.436249
5     3.317605
6     5.038168
7     3.151333
8     2.550557
9     3.253082
10    1.985489
11    1.633010
12    1.721207
13    1.619174
14    1.405890
15    1.401980
16    1.352102
17    1.212956
18    1.127188
19    1.105252
20    1.179168
21    1.174938
22    1.057719
23    0.950536
24    0.999303
25    1.002766
26    1.029674
27    0.932618
28    0.927254
29    0.932435
30    0.957261
31    0.866608
32    0.848202
33    0.850335
34    0.844588
35    0.798686
36    0.798506
37    0.827807
38    0.820418
39    0.777891
40    0.786068
41    0.796943
42    0.747810
43    0.745099
44    0.740252
45    0.749806
46    0.744475


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.transductive_training.mapped_triples,
        additional_filter_triples=[
        dataset.transductive_training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/6.72k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.007477   0.007269     0.007170
hits_at_1                     0.001190   0.001190     0.001190
hits_at_3                     0.003571   0.003571     0.003571
hits_at_5                     0.004911   0.004911     0.004911
hits_at_10                    0.011310   0.011310     0.011310
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.021360   0.021330     0.021302
hits_at_1                     0.002083   0.002083     0.002083
hits_at_3                     0.007738   0.007738     0.007738
hits_at_5                     0.014583   0.014583     0.014583
hits_at_10                    0.029613   0.029613     0.029613
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.014418   0.014300     0.014236
hits_at_1                     0.001637   0.001637     0.001637
hits_at_3                     0.005655  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/895 [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.015885   0.015521     0.015295
hits_at_1                     0.002235   0.002235     0.002235
hits_at_3                     0.013408   0.013408     0.013408
hits_at_5                     0.015642   0.015642     0.015642
hits_at_10                    0.025698   0.025698     0.025698
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.037220   0.037163     0.037108
hits_at_1                     0.005587   0.005587     0.005587
hits_at_3                     0.017877   0.017877     0.017877
hits_at_5                     0.029050   0.029050     0.029050
hits_at_10                    0.070391   0.070391     0.070391
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.026552   0.026342     0.026202
hits_at_1                     0.003911   0.003911     0.003911
hits_at_3                     0.015642  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/1.14k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.015468   0.015114     0.014892
hits_at_1                     0.003515   0.003515     0.003515
hits_at_3                     0.008787   0.008787     0.008787
hits_at_5                     0.013181   0.013181     0.013181
hits_at_10                    0.025483   0.025483     0.025483
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.040752   0.040683     0.040617
hits_at_1                     0.009666   0.009666     0.009666
hits_at_3                     0.020211   0.020211     0.020211
hits_at_5                     0.028998   0.028998     0.028998
hits_at_10                    0.066784   0.066784     0.066784
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.028110   0.027899     0.027755
hits_at_1                     0.006591   0.006591     0.006591
hits_at_3                     0.014499  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['aggregation'] = 'mlp'

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   
22                     aggregation   

                                                value  
0                              

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)